In [1]:
import pandas as pd 
import numpy as np
import cv2
import os
from utils.detectFace import detect

In [19]:
from ultralytics import YOLO
model = YOLO("Models/yolov11n-face.pt")
data_path = 'data'
bbox = []
for person_name in os.listdir(data_path):
    person_path = os.path.join(data_path, person_name)
    if not os.path.isdir(person_path):
        continue
    for image in os.listdir(person_path):
        image_path = os.path.join(person_path, image)
        frame = cv2.imread(image_path)
        if frame is None:
            continue
        boxes, bounding_box_list = detect(model, frame)
        bbox.append(bounding_box_list)
        print(f"Processing {image_path} with {len(bounding_box_list)} faces detected.")
        print(f"Processed {image_path}")
print("Done!")


0: 416x640 1 face, 108.9ms
Speed: 6.3ms preprocess, 108.9ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 640)
Processing data/Messi/pic7.png with 1 faces detected.
Processed data/Messi/pic7.png

0: 480x640 1 face, 89.5ms
Speed: 7.8ms preprocess, 89.5ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
Processing data/Messi/pic10.png with 1 faces detected.
Processed data/Messi/pic10.png

0: 384x640 1 face, 72.6ms
Speed: 1.5ms preprocess, 72.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Processing data/Messi/pic5.png with 1 faces detected.
Processed data/Messi/pic5.png

0: 640x448 1 face, 88.4ms
Speed: 2.0ms preprocess, 88.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 448)
Processing data/Messi/pic8.png with 1 faces detected.
Processed data/Messi/pic8.png

0: 384x640 1 face, 64.6ms
Speed: 2.3ms preprocess, 64.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Processing data/Messi/pic9.png with 1 fa

In [31]:
for box in bbox[:5]:
    x1, y1, x2, y2 = box[0]
    print(f"Bounding box: ({x1}, {y1}), ({x2}, {y2})")

Bounding box: (359, 86), (462, 249)
Bounding box: (505, 139), (739, 476)
Bounding box: (136, 19), (162, 60)
Bounding box: (232, 283), (796, 1081)
Bounding box: (116, 23), (167, 93)


In [35]:
# Lưu vào file csv
output_path = 'output/bounding_boxes.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Tạo list lưu tất cả bounding box với thông tin đầy đủ
rows = []
for person_name in os.listdir(data_path):
    person_path = os.path.join(data_path, person_name)
    if not os.path.isdir(person_path):
        continue
    for image in os.listdir(person_path):
        image_path = os.path.join(person_path, image)
        for box in bbox:
            x, y, w, h = box[0]
            rows.append([person_name, image_path, x1, y1, x2, y2])

df = pd.DataFrame(rows, columns=['person', 'image', 'x', 'y', 'w', 'h'])
df.to_csv(output_path, index=False)

In [36]:
data = pd.read_csv(output_path)

In [37]:
data = data.drop_duplicates(subset=['person', 'image', 'x', 'y', 'w', 'h'])
data.to_csv(output_path, index=False)

In [45]:
from utils.face_regconition import cal_similar
import pandas as pd
import numpy as np
from utils.face_regconition import cal_similar, convert_str2array

# Đọc file embedding database
embedding_df = pd.read_csv("output/embeddings.csv")

# Giả sử bạn có embedding khuôn mặt mới (vectorC)
# Ví dụ: lấy embedding của một người bất kỳ trong database
vectorC = convert_str2array(embedding_df.iloc[10, 1])  # embedding ở cột 1

# Tính toán nhận diện
person_name = cal_similar(vectorC, embedding_df)
print(vectorC)
print("Kết quả nhận diện:", person_name)

[   0.053519     0.26432      0.2396 ...     0.40677           0       1.065]
Kết quả nhận diện: Dat
